In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from datetime import datetime
pd.options.mode.chained_assignment=None
import tsfresh
from dtaidistance import dtw
from sklearn.preprocessing import normalize
import seaborn as sns
import os
import re

### hampel

In [ ]:

def  hampel(data,k=5,t0=3):
 #1判断长度问题
    if len(data) == 0:
        print('长度为0')
        return False
    data = data.loc[:,['TimeStr', 'Ia']]
    data.loc[:,'TimeStr'] = pd.to_datetime(data['TimeStr'])
    data.reset_index(drop=True, inplace=True)
    #1删除重复值
    data.loc[:,'TimeStr']=pd.to_datetime(data['TimeStr'].apply(lambda x: datetime.strftime(x, format='%Y-%m-%d %H:%M:%S')))
    data.drop_duplicates('TimeStr', inplace=True)
    data.reset_index(drop=True, inplace=True)
     #2补全时间 
    time_series = pd.date_range(data["TimeStr"].min(), data["TimeStr"].max(), freq='s')
    ts = pd.DataFrame(time_series, columns=["time"])
    data = data.merge(ts, how='right', left_on="TimeStr", right_on="time")
    data.drop(['TimeStr'], axis=1, inplace=True)
    data.sort_values(by='time', inplace=True)
    data.reset_index(drop=True, inplace=True)
    Ia = data['Ia']
    #3Hampel Filter
    L = 1.4826
    rolling_median=Ia.rolling(window=k,min_periods=1,center=True).median()
    difference=np.abs(rolling_median-Ia)
    median_abs_deviation=difference.rolling(window=k,min_periods=1,center=False).median()
    threshold=t0*L*median_abs_deviation
    outlier_idx=difference>threshold
    Ia[outlier_idx]=rolling_median[outlier_idx]
    #4补缺失值
    Ia.fillna(0,inplace=True)  #check
    if len(Ia) == 86401:
        return Ia
    else:
        return False
    
def mkdir_plot(path):
    isExists=os.path.exists(path)
    if not isExists:
        os.mkdir(path)
        return True
    else:
        return True

### feature

In [ ]:
class feature(object):
    def __init__(self,Ia,l):
        self.l=l
        self.max_window=60*30
        self.len_window=60*l 
        self.interval=60
#         self.data=hampel(data)
        self.data = Ia.copy()
        self.y={}
        self.matrix_length=int((len(self.data)-self.max_window)/self.interval)+1 #特征向量的行数 或 窗口的总个数
        self.featue_matrix=pd.DataFrame({'window_no':range(self.matrix_length)})

        #self.y 字典 ：存储所有滑窗的y
    def data_process(self):
        for i in range(self.matrix_length):
#             self.data_y.setdefault(i)
            self.y.setdefault(i)
            idx_start=i*self.interval
            idx_end=idx_start + self.len_window
            if type(self.data) == pd.Series:
                self.y[i]=self.data[idx_start:idx_end].reset_index(drop=True)

        return self
        #遍历self.y 计算对应的特征向量
    
    def raw_y(self):
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(self.y[i][0])
        self.featue_matrix['raw_y_%s' % self.l]=res_l
        
        return self
        
        
    def longest_strike_above_mean(self):
        f=tsfresh.feature_extraction.feature_calculators.longest_strike_above_mean
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i].reset_index(drop=True)))
        self.featue_matrix['longest_strike_above_mean_%s' % self.l]=res_l
        
        return self
        

    def sum_values(self):
        f=tsfresh.feature_extraction.feature_calculators.sum_values
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['sum_values_%s' % self.l]=res_l

        return self
    
    def ratio_beyond_r_sigma(self,r=3):
        f=tsfresh.feature_extraction.feature_calculators.ratio_beyond_r_sigma
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],r))
        self.featue_matrix['ratio_beyond_r_sigma_%s' % self.l]=res_l

        return self
    
    def absolute_sum_of_changes(self,r=0.3):
        f=tsfresh.feature_extraction.feature_calculators.absolute_sum_of_changes
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['absolute_sum_of_changes_%s' % self.l]=res_l

        return self
    
    def mean_second_derivative_central(self):
        f=tsfresh.feature_extraction.feature_calculators.mean_second_derivative_central
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['mean_second_derivative_central_%s' % self.l]=res_l
          
        return self
    
    def mean_(self):
        f=np.mean
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['mean_%s' % self.l]=res_l
        return self

    def quantile_25(self,q=0.25):
        f = tsfresh.feature_extraction.feature_calculators.quantile
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],q))
        self.featue_matrix['quantile_25_%s' % self.l]=res_l
        return self
    
    def quantile_75(self,q=0.75):
        f = tsfresh.feature_extraction.feature_calculators.quantile
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],q))
        self.featue_matrix['quantile_75_%s' % self.l]=res_l
        return self
        
    def variance(self):
        f=tsfresh.feature_extraction.feature_calculators.variance
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['variance_%s' % self.l]=res_l
        return self
  
    def linear_trend(self):
        param = [{"attr":"slope"}]
        f = tsfresh.feature_extraction.feature_calculators.linear_trend
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],param)[0][1])
        self.featue_matrix['linear_trend_%s' % self.l]=res_l
        return self

    def median_(self):
        f = tsfresh.feature_extraction.feature_calculators.median
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['median_%s' % self.l]=res_l
        return self
    
    def kurtosis(self):
        f = tsfresh.feature_extraction.feature_calculators.kurtosis
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['kurtosis_%s' % self.l]=res_l
        return self
    
    def maximum(self):
        f = tsfresh.feature_extraction.feature_calculators.maximum
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['maximum_%s' % self.l]=res_l
        return self
    
    def range_count_8(self,min=0.0001,max=8):
        f = tsfresh.feature_extraction.feature_calculators.range_count
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],min,max))
        self.featue_matrix['range_count_8_%s' % self.l]=res_l
        return self
    
    def range_count_20(self,min=8.0001,max=20):
        f = tsfresh.feature_extraction.feature_calculators.range_count
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],min,max))
        self.featue_matrix['range_count_20_%s' % self.l]=res_l
        return self
    
    def range_count_50(self,min=20.0001,max=50):
        f = tsfresh.feature_extraction.feature_calculators.range_count
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],min,max))
        self.featue_matrix['range_count_50_%s' % self.l]=res_l
        return self
    
    def percentage_of_reoccurring_datapoints_to_all_datapoints(self):
        f = tsfresh.feature_extraction.feature_calculators.percentage_of_reoccurring_datapoints_to_all_datapoints  
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.featue_matrix['percentage_of_reoccurring_datapoints_to_all_datapoints_%s' % self.l]=res_l
        return self
    
    def standardscaler(self): #不包括dtw  输出所有的特征列，忽略第一列索引列
        f=StandardScaler()
        out=f.fit_transform(self.featue_matrix.iloc[:,1:])
        
        return out

     

In [ ]:
def construct_case(Ia,l):
    '''
    输入数据data window长度l
    输出特征矩阵 nrow=l,ncol=特征个数
    '''
    case=feature(Ia,l)  #默认取 5 10 20 30分钟 四种滑窗 
    case.data_process()
    case.raw_y()
    case.longest_strike_above_mean()
    case.sum_values()
    case.ratio_beyond_r_sigma() #异常点
    case.absolute_sum_of_changes()
    case.mean_second_derivative_central()
    case.mean_()
    case.quantile_25()
    case.quantile_75()
    case.variance()
    case.linear_trend()
    case.median_()
    case.kurtosis()
    case.maximum()  
    case.range_count_8()
    case.range_count_20()
    case.range_count_50()
    case.percentage_of_reoccurring_datapoints_to_all_datapoints()
    
    return case.featue_matrix



In [ ]:
def get_result(cur_Ia,class_name,uuid):
    feature_5=construct_case(cur_Ia,5)
    print('start to make feature_5')
    feature_10 = construct_case(cur_Ia,10)
    print('start to make feature_10')
    feature_20 = construct_case(cur_Ia,20)
    print('start to make feature_20')
    feature_30 = construct_case(cur_Ia,30)
    print('start to make feature_30')
#     feature_5.to_pickle('/home/wrj/FeatureDT/result/%s_%s_feature_5.pkl' % (class_name,uuid))
#     feature_10.to_pickle('/home/wrj/FeatureDT/result/%s_%s_feature_10.pkl' % (class_name,uuid))
#     feature_20.to_pickle('/home/wrj/FeatureDT/result/%s_%s_feature_30.pkl' % (class_name,uuid))
#     feature_30.to_pickle('/home/wrj/FeatureDT/result/%s_%s_feature_60.pkl' % (class_name,uuid))
    
    return feature_5,feature_10,feature_20,feature_30

In [ ]:
def merge_result(feature_5,feature_10,feature_20,feature_30,datatype,machine,uuid):
    variable_datasets = ['除尘器', '火焰切割机', '电源',  '主机', '激光切割机']
    y_label_dict = {'除尘器':1, '火焰切割机':2, '电源':3,  '主机':4, '激光切割机':5}
    data_merge = pd.concat((feature_5,feature_10,feature_20,feature_30),axis=1)
    data_merge.dropna(inplace=True)
    cur_label = y_label_dict[machine]
    data_merge['label'] = np.array([cur_label]*len(data_merge))
    data_merge.drop(["raw_y_10","raw_y_20","raw_y_30"],axis=1,inplace=True )
    data_merge.drop(['window_no'],axis=1,inplace=True)
    data_merge.to_pickle('/home/wrj/FeatureDT/result/%s/%s_%s_merge.pkl' % (datatype,machine,uuid))
    print("merge已经写入目标地址")
    print("data_merge shape :",data_merge.shape)
    
    return data_merge

#### X

In [ ]:
%%time
def get_feature_matrix(datatype):
    path = r'/home/wrj/FeatureDT/%s' % datatype
    variable_datasets = ['除尘器', '火焰切割机', '电源',  '主机', '激光切割机']
    y_label_dict = {'除尘器':1, '火焰切割机':2, '电源':3,  '主机':4, '激光切割机':5}
    uid_list = []
    machine_list = []
    for machine in variable_datasets:
        read_path = os.path.join(path,machine)
        variable_file = os.listdir(read_path)
    #     print(variable_file)
        for i,file in enumerate(variable_file):
            file_path = os.path.join(read_path,file)
            print(i,file_path)
            uuid = file[0:-4]
            uid_list.append(uuid)
            machine_list.append(machine)
            cur_data = pd.read_csv(file_path)
            cur_Ia = hampel(cur_data)
            if type(cur_Ia) is not bool:
                feature_5,feature_10,feature_20,feature_30 = get_result(cur_Ia,machine,uuid)
                data_merge = merge_result(feature_5,feature_10,feature_20,feature_30,datatype,machine,uuid)
            else:
                print("数据有问题")
    return feature_5,feature_10,feature_20,feature_30,data_merge


### Data

In [ ]:
def acquire_data(datatype):
    path = '/home/wrj/FeatureDT/result/%s' % datatype
    merge_list = os.listdir(path)
    for i in range(len(merge_list)):
        read_merge_file = os.path.join(path,merge_list[i])
        print(i,read_merge_file)
        if i == 0:
            cur_data_0 = pd.read_pickle(read_merge_file)
        elif i == 1:
            cur_data_1 = pd.read_pickle(read_merge_file)
            temp_data = pd.concat((cur_data_0,cur_data_1),axis = 0)
        else:
            cur_data = pd.read_pickle(read_merge_file)
            temp_data = pd.concat((temp_data,cur_data),axis = 0)
    temp_data.reset_index(drop=True,inplace=True)
    temp_data.to_pickle('/home/wrj/FeatureDT/model/data/temp_%s.pkl' % datatype)
    
    return temp_data


In [ ]:
%%time
feature_5,feature_10,feature_20,feature_30,data_merge = get_feature_matrix("train")

In [ ]:
%%time
feature_5,feature_10,feature_20,feature_30,data_merge = get_feature_matrix("test")

In [ ]:
dtest = acquire_data("train")

In [ ]:
dtest = acquire_data("test")